# Carregar modelo

In [ ]:
from keras.models import load_model

modelo = load_model('models/CNNmodel.h5')
modelo.summary()

# Teste Finaceiro

In [15]:
from json import dumps
from tqdm import tqdm
import numpy as np
from obterDados import obterSimboloPosicao, obterSimboloData

target = 30
stop = 10
timeLimit = 24
total = 0
long = False
short = False
position = 0
candle = -1
trade_summary = {
    'longSuccess' : [0, 0],
    'longFail' : [0, 0],
    'longEndSuccess' : [0, 0],
    'longEndFail' : [0, 0],
    'shortSuccess' : [0, 0],
    'shortFail' : [0, 0],
    'shortEndSuccess' : [0, 0],
    'shortEndFail' : [0, 0],
}
buys_and_sells = []
for i in tqdm(range(50*108, -1, -1)):
    histM5 = obterSimboloPosicao('WDO$N', n=300, delayCandles=i)
    histM5['minute'] = histM5.index.minute
    histM5['hour'] = histM5.index.hour
    histM5['day_of_week'] = histM5.index.day_of_week
    histM5['day'] = histM5.index.day
    histM5NP = histM5.to_numpy()
    hist = np.zeros((501, 6))
    hist[0, -4:] = histM5NP[-1][-4:]
    hist[1:301] = histM5NP[:, :6]
    histD1 = obterSimboloData(histM5.index[-1])
    histD1NP = histD1.to_numpy()
    hist[301:] = histD1NP
    vmax = hist[1:301, :4].max()
    vmin = hist[1:301, :4].min()
    hist[1:301, :4] = (hist[1:301, :4] - vmin) / (vmax - vmin)
    vmax = hist[1:301, 4].max()
    vmin = hist[1:301, 4].min()
    hist[1:301, 4] = (hist[1:301, 4] - vmin) / (vmax - vmin)
    vmax = hist[1:301, 5].max()
    vmin = hist[1:301, 5].min()
    hist[1:301, 5] = (hist[1:301, 5] - vmin) / (vmax - vmin)
    vmax = hist[301:, :4].max()
    vmin = hist[301:, :4].min()
    hist[301:, :4] = (hist[301:, :4] - vmin) / (vmax - vmin)
    vmax = hist[301:, 4].max()
    vmin = hist[301:, 4].min()
    hist[301:, 4] = (hist[301:, 4] - vmin) / (vmax - vmin)
    vmax = hist[301:, 5].max()
    vmin = hist[301:, 5].min()
    hist[301:, 5] = (hist[301:, 5] - vmin) / (vmax - vmin)
    hist[0, 2] /= 60
    hist[0, 3] /= 24
    hist[0, 4] /= 4
    hist[0, 5] /= 31
    priceNow = histM5['close'].iloc[-1]
    if long:
        if priceNow >= position + target:
            total += priceNow - position
            long = False
            trade_summary['longSuccess'][0] += 1
            trade_summary['longSuccess'][1] += priceNow - position
        if priceNow <= position - stop:
            total += priceNow - position
            long = False
            trade_summary['longFail'][0] += 1
            trade_summary['longFail'][1] += priceNow - position
        if i >= candle + timeLimit:
            total += priceNow - position
            long = False
            if priceNow - position > 0:
                trade_summary['longEndSuccess'][0] += 1
                trade_summary['longEndSuccess'][1] += priceNow - position
            else:
                trade_summary['longEndFail'][0] += 1
                trade_summary['longEndFail'][1] += priceNow - position
        continue
    if short:
        if priceNow <= position - target:
            total += position - priceNow
            short = False
            trade_summary['shortSuccess'][0] += 1
            trade_summary['shortSuccess'][1] += position - priceNow
        if priceNow >= position + stop:
            total += position - priceNow
            short = False
            trade_summary['shortFail'][0] += 1
            trade_summary['shortFail'][1] += position - priceNow
        if i >= candle + timeLimit:
            total += position - priceNow
            short = False
            if position - priceNow > 0:
                trade_summary['shortEndSuccess'][0] += 1
                trade_summary['shortEndSuccess'][1] += position - priceNow
            else:
                trade_summary['shortEndFail'][0] += 1
                trade_summary['shortEndFail'][1] += position - priceNow
        continue

    previsao = np.argmax(modelo.predict(np.array([hist]), verbose=0)[0])
    if previsao == 1:
        long = True
        position = priceNow
        candle = i
        buys_and_sells.append([i, 'Buy', priceNow])
    if previsao == 2:
        short = True
        position = priceNow
        candle = i
        buys_and_sells.append([i, 'Sell', priceNow])

print('Total:', total)
# print('Operações: ', dumps(trade_summary, indent=4))

100%|██████████| 5401/5401 [01:39<00:00, 54.40it/s]

Total: 284.0


In [19]:
from matplotlib import pyplot as plt

plt.figure(figsize=(200, 50))
plt.plot(obterSimboloPosicao('WDO$N', n=50*108)['close'].to_numpy())
for idx, action, price in buys_and_sells:
    if action == 'Buy':
        plt.plot(-idx+50*108-1, price, 'o', color='g')
    if action == 'Sell':
        plt.plot(-idx+50*108-1, price, 'o', color='r')
plt.show()